In [53]:
import pandas as pd
import requests

In [80]:
geo_data = pd.read_csv('geo-by-area.csv')
session = requests.Session()

In [55]:
from json import JSONDecodeError


def get_voice_data(session, vp):
    try:
        data = session.get(f'http://partizan-results.com/data/results/byStation/{vp}.json').json()
    except JSONDecodeError:
        return None
    
    data.pop('protocolPhotos')
    choices = data.pop('choices')
    for candidate in (
        'against', 'cherechen', 'corrupted', 'dmitriyev', 'ignore', 'kanopatskaja', 'lukashenko', 'tihanovkaja'
    ):
        for field in ('registered', 'officialVotes', 'photoVoices'):
            data[f'{candidate}_{field}'] = (
                int(choices[candidate][field]) 
                if choices[candidate][field] is not None 
                else None 
            )
    
    return data

In [103]:
full_data = []

In [107]:
for record in geo_data[5740:].iterrows():
    print(record[0])
    voice_data = get_voice_data(session, record[1]['id'])
    
    if voice_data is not None:
        voice_data['id'] = record[1]['id']
        full_data.append(voice_data)

5740
5741
5742
5743
5744
5745
5746
5747
5748
5749
5750
5751
5752
5753
5754
5755
5756
5757
5758
5759
5760
5761
5762
5763
5764
5765
5766


In [108]:
df = pd.DataFrame(full_data, dtype=pd.Int64Dtype)
df = df[[df.columns[-1]] + list(df.columns[:-1])]
df.to_csv('voice.csv')

In [111]:
df = pd.read_csv('voice.csv', converters={x: lambda z: int(z) if z else z  for x in df.columns[2:]})

In [23]:
df[df.columns[1:]].to_csv('voice.csv', index=False)

In [34]:
failed_geo = geo_data[geo_data['id'].isin(set(geo_data['id']) - set(df['id']))]

In [68]:
emb_data = []
for vid in [x for x in failed_geo['id'] if  x > '07-00']:
    fixed_id = '07-102-' + vid.split('-')[-1]
    
    print(fixed_id)
    
    vdata = get_voice_data(session, fixed_id)
    if vdata is None:
        raise ValueError(vid)
        
    vdata['id'] = vid
    emb_data.append(vdata)

07-102-0075
07-102-0076
07-102-0077
07-102-0078
07-102-0079
07-102-0080
07-102-0081
07-102-0082
07-102-0083
07-102-0084
07-102-0085
07-102-0086
07-102-0087
07-102-0088
07-102-0089
07-102-0090
07-102-0091
07-102-0092
07-102-0093
07-102-0094
07-102-0095
07-102-0096
07-102-0097
07-102-0098
07-102-0099
07-102-0100
07-102-0101
07-102-0102
07-102-0103
07-102-0104
07-102-0105
07-102-0106
07-102-0107
07-102-0108
07-102-0109
07-102-0110
07-102-0111
07-102-0112
07-102-0113
07-102-0114
07-102-0115
07-102-0116
07-102-0117
07-102-0118


In [72]:
emb_df = pd.DataFrame(emb_data, dtype=pd.Int64Dtype)
emb_df = emb_df[[emb_df.columns[-1]] + list(emb_df.columns[:-1])]

In [76]:
emb_df.to_csv('voice.csv', mode='a', header=False, index=False)

In [129]:
voice_with_missing.to_csv('voice.csv', index=False)